# Objet : Analyse des relations entre champs des données IRVE

## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par la détection d'incohérences de relations
- l'analyse a posteriori permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 2% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(active link on jupyter Notebook or Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Analyse des relations entre champs](#3---Analyse-des-relations-entre-champs)
- [4 - Contrôle des relations](#4---Contrôle-des-relations)
- [5 - Contrôle du schéma de données](#5---Contrôle-du-schéma-de-données)
- [6 - Exemples d'analyse de relations](#6---Exemples-d'analyse-de-relations)
- [Annexe : Initialisation du modèle de données 'mermaid'](#Annexe-:-Initialisation-du-modèle-de-données-'mermaid')

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"
_________________

# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ associé à une ligne du tableau*

In [3]:
# la variable 'irve' est définie en fin de Notebook
mermaid(irve)

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [52]:
from collections import Counter
from observation import Ilist, Iindex, Analysis
import pandas as pd
res = {}
chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'
res['date'] = '2023-03-03'
file = 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv'
data = pd.read_csv(chemin + file, sep=',', low_memory=False)
res['len_data'] = len(data)
print('data : ', res['len_data']) 

data :  51276


In [66]:
data3 = data.iloc[:, 0:41]
data3[['id_station_itinerance','id_pdc_itinerance']] = data3[['id_station_itinerance','id_pdc_itinerance']].astype('string')
data3['id'] = list(range(len(data3)))
#print(data3.dtypes)

----
## initialisation de l'outil d'analyse
L'outil d'analyse ([voir présentation](https://github.com/loco-philippe/Environmental-Sensing/blob/main/documentation/FR_tabular_structure.ipynb)) est chargé à partir des données Pandas :
On dénombre 211 370 valeurs différentes sur un total de 51 276 x 50 valeurs ("taux d'unicité" de 8.2%)

In [53]:
idxs = Ilist.obj(data)
res['lignes_data'] = len(idxs)
res['champs_data'] = len(idxs.idxlen)
res['unique_data'] = sum(idxs.idxlen)
print('nombre de lignes              : ', res['lignes_data'])
print('nombre de champs              : ', res['champs_data'])
print('nombre de valeurs différentes : ', res['unique_data'])

nombre de lignes              :  51276
nombre de champs              :  50
nombre de valeurs différentes :  211370


_________________
# 3 - Analyse des relations entre champs
## Arborescence des champs
- l'examen de "l'arbre de dérivation" indique un faible niveau d'interdépendance entre champs (seuls quelques-uns ne sont pas liés à la racine). 
- Par exemple les champs consolidated_longitude(44) et consolidated_latitude(45) sont bien dérivés du champ coordonneesXY(13)


In [6]:
print(idxs.tree())
# valeur entre parenthèses : nombre de valeurs distinctes

-1: root-derived (51276)
   0 : nom_amenageur (2474)
   1 : siren_amenageur (1182)
   2 : contact_amenageur (548)
   3 : nom_operateur (237)
   4 : contact_operateur (274)
   5 : telephone_operateur (357)
   6 : nom_enseigne (1493)
   7 : id_station_itinerance (18544)
   8 : id_station_local (10731)
   9 : nom_station (15463)
   10: implantation_station (5)
   11: adresse_station (16496)
   12: code_insee_commune (5242)
   13: coordonneesXY (17241)
      44: consolidated_longitude (17069)
      45: consolidated_latitude (17018)
   14: nbre_pdc (50)
   15: id_pdc_itinerance (42457)
   16: id_pdc_local (27186)
   17: puissance_nominale (87)
   18: prise_type_ef (8)
   19: prise_type_2 (8)
   20: prise_type_combo_ccs (8)
   21: prise_type_chademo (7)
   22: prise_type_autre (7)
   23: gratuit (8)
   24: paiement_acte (8)
   25: paiement_cb (7)
   26: paiement_autre (8)
   27: tarification (212)
   28: condition_acces (2)
   29: reservation (7)
   30: horaires (516)
   31: accessibilite_pm

# 4 - Contrôle des relations
Le contrôle consiste à valider les relations inter entités ainsi que les relations intra entités. Pour le modèle identifié précédemment, les contrôles sont les suivants :
- contrôle d'unicité des lignes (une ligne est associée à un point de charge)
- contrôle inter entités (pdc-station, station-opérateur, station-localisation, station-enseigne)
- contrôle intra entités (station, localisation)

## unicité des lignes
- vérification de la relation 1-1 entre le champ 'root' (liste des lignes) et le champ 'id_pdc_itinerance' (cf analyse au chapitre 5)
- le nombre de lignes en écart est de 16642 (avec historique) et 1068 (hors historique des modifications)

In [7]:
champ = idxs.nindex
root = Iindex.ext(list(range(len(idxs))))

ecart_root = champ('id_pdc_itinerance').coupling(root, reindex=True)

print('lignes en écart : ',len(ecart_root))

lignes en écart :  16642


## relations inter-entités
L'analyse est effectuée sur la base des données hors historique (data2).    
Les relation 1-n avec les points de charge n'ont pas besoin d'être controlées (unicité des points de charge).
   
Nota :
- *si on transforme la relation 1-n entre station et localisation en une relation 1-1, les écarts sont beaucoups plus importants (plus de 7 000 lignes), ceci indique qu'à une même localisation on trouve plusieurs stations (ex. identifiants de points de charges utilisés comme identifiants de stations)*

In [8]:
data2 = data.sort_values(by='last_modified').drop_duplicates('id_pdc_itinerance', keep='last').reset_index(drop=True)
idxs2 = Ilist.obj(data2)
champ = idxs2.nindex

In [9]:
ecart_oper = champ('id_station_itinerance').coupling(champ('contact_operateur'), reindex=True)
ecart_XY = champ('id_station_itinerance').coupling(champ('coordonneesXY'), reindex=True)
ecart_ens = champ('id_station_itinerance').coupling(champ('nom_enseigne'), reindex=True)

print('écart relation id_station - contact_operateur : ', len(ecart_oper), ' lignes')
print('écart relation id_station - coordonneesXY     : ', len(ecart_XY), ' lignes')
print('écart relation id_station - nom_enseigne      : ', len(ecart_ens), ' lignes')

écart relation id_station - contact_operateur :  285  lignes
écart relation id_station - coordonneesXY     :  705  lignes
écart relation id_station - nom_enseigne      :  547  lignes


## relations intra entités
On ne considère que les champs obligatoires qui doivent avoir une relation 1-n avec le champ 'id_station_itinerance' pour les stations, 'coordonneesXY' pour les localisations.

In [10]:
ecart_nom = champ('id_station_itinerance').coupling(champ('nom_station'), reindex=True)
ecart_imp = champ('id_station_itinerance').coupling(champ('implantation_station'), reindex=True)
ecart_nbr = champ('id_station_itinerance').coupling(champ('nbre_pdc'), reindex=True)
ecart_acc = champ('id_station_itinerance').coupling(champ('condition_acces'), reindex=True)
ecart_hor = champ('id_station_itinerance').coupling(champ('horaires'), reindex=True)
ecart_rou = champ('id_station_itinerance').coupling(champ('station_deux_roues'), reindex=True)

ecart_adr = champ('coordonneesXY').coupling(champ('adresse_station'), reindex=True)


print('écart relation id_station - nom_station          : ',len(ecart_nom), ' lignes')
print('écart relation id_station - implantation_station : ',len(ecart_imp), ' lignes')
print('écart relation id_station - nbre_pdc             : ',len(ecart_nbr), ' lignes')
print('écart relation id_station - condition_acces      : ',len(ecart_acc), '   lignes')
print('écart relation id_station - horaires             : ',len(ecart_hor), ' lignes')
print('écart relation id_station - station_deux_roues   : ',len(ecart_rou), ' lignes', '\n')

print('écart relation coordonneesXY - adresse_station   : ',len(ecart_adr), ' lignes')

écart relation id_station - nom_station          :  288  lignes
écart relation id_station - implantation_station :  217  lignes
écart relation id_station - nbre_pdc             :  597  lignes
écart relation id_station - condition_acces      :  5    lignes
écart relation id_station - horaires             :  263  lignes
écart relation id_station - station_deux_roues   :  616  lignes 

écart relation coordonneesXY - adresse_station   :  604  lignes


# 5 - Contrôle du schéma de données

Les contrôles précédents peuvent être réalisés à partir de la propriété 'relationship' définie dans le schéma de données.

## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [11]:
fields = [

 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},

 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

## Analyse et résultats
L'analyse fournit la liste des enregistrements incohérents pour chaque propriété.
Des champs booléens (valeur False à chaque enregistrement incohérent) pour chaque propriété sont ajoutés au jeu de données.

In [54]:
# génération du dictionnaire contenant la liste des enregistrements incohérent par propriété
analyse = Analysis(data)
dic_res = analyse.check_relationship(fields)

# ajout d'un champ pour chaque propriété contrôlée (plus deux champs de synthèse)
data['total'] = True
data['somme'] = 0
for name, lis in dic_res.items():
    data[name] = True
    data.loc[lis, name] = False
    data['total'] = data['total'] & data[name]
    data['somme'] = data['somme'] - data[name] + 1
    print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
maxi = max(data['somme'])
df_max = data[data.somme == maxi]

# résultats
print("nombre d'enregistrements avec au moins une erreur : ", len(data) - data.total.sum())
print("nombre d'enregistrements avec ", maxi, " erreurs : ", len(df_max))
df_max.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY')]

contact_operateur - id_station_itinerance          11311
nom_enseigne - id_station_itinerance                8108
coordonneesXY - id_station_itinerance              12417
id_station_itinerance - id_pdc_itinerance           1097
nom_station - id_station_itinerance                 2459
implantation_station - id_station_itinerance        1837
nbre_pdc - id_station_itinerance                    2050
condition_acces - id_station_itinerance              627
horaires - id_station_itinerance                   10463
station_deux_roues - id_station_itinerance         11560
adresse_station - coordonneesXY                     1403
nombre d'enregistrements avec au moins une erreur :  16288
nombre d'enregistrements avec  11  erreurs :  25


,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY
765,Non concerné,Non concerné,WICKER TP BUREAUX,"[7.56065,48.73133]"
803,Non concerné,Non concerné,WICKER TP ATELIERS,"[7.56065,48.73133]"
1185,Non concerné,Non concerné,Île-aux-moines - Embarcadère,"[-2.852219, 47.601017]"
1211,Non concerné,Non concerné,Île-aux-moines - Embarcadère,"[-2.852219, 47.601017]"
1231,Non concerné,Non concerné,Île-aux-moines - Embarcadère,"[-2.852219, 47.601017]"
1337,Non concerné,Non concerné,Terminal Cherbourg,"[-1.605048, 49.644253]"
1383,Non concerné,Non concerné,Bananes vertes,"[-61.69, 16.03]"
1384,Non concerné,Non concerné,SEDRFGHJKL,"[-61.69, 16.03]"
2072,Non concerné,Non concerné,AUBERGE SAINT WALFRID,"[7.04449, 49.114638]"
2073,Non concerné,Non concerné,LA MEULE DE PIERRE,"[-1.605048, 49.644253]"


In [22]:
# génération du dictionnaire contenant la liste des enregistrements incohérent par propriété
analyse = Analysis(data2)
dic_res = analyse.check_relationship(fields)

# ajout d'un champ pour chaque propriété contrôlée (plus deux champs de synthèse)
data2['total'] = True
data2['somme'] = 0
for name, lis in dic_res.items():
    data2[name] = True
    data2.loc[lis, name] = False
    data2['total'] = data2['total'] & data2[name]
    data2['somme'] = data2['somme'] - data2[name] + 1
    print('{:<50} {:>5}'.format(name, len(data2) - data2[name].sum()))
maxi = max(data2['somme'])
df_max = data2[data2.somme == maxi]

# résultats
print("nombre d'enregistrements avec au moins une erreur : ", len(data2) - data2.total.sum())
print("nombre d'enregistrements avec ", maxi, " erreurs : ", len(df_max))
df_max.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY')]

contact_operateur - id_station_itinerance            285
nom_enseigne - id_station_itinerance                 547
coordonneesXY - id_station_itinerance                705
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  288
implantation_station - id_station_itinerance         217
nbre_pdc - id_station_itinerance                     597
condition_acces - id_station_itinerance                5
horaires - id_station_itinerance                     263
station_deux_roues - id_station_itinerance           616
adresse_station - coordonneesXY                      604
nombre d'enregistrements avec au moins une erreur :  2218
nombre d'enregistrements avec  9  erreurs :  14


,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY
7923,FRE10E40150,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7924,FRE10E40151,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7925,FRE10E40149,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7926,FRE10E40152,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7927,FRE10E40154,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7928,FRE10E40155,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7929,FRE10E40148,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
7930,FRE10E40153,Non concerné,HERVE THERMIQUE JOUE LES TOURS,"[0.654826, 47.353958]"
41082,FRLDLE00001763,Non concerné,LIDL ST AUBIN SUR SCIE,"[0.9605582,49.5355585]"
41083,FRLDLE00001764,Non concerné,LIDL ST AUBIN SUR SCIE,"[0.9605582,49.5355585]"


In [23]:
df_non_concerne = data2[data2.id_station_itinerance == 'Non concerné']
df_concerne = data2[data2.id_station_itinerance != 'Non concerné']
print("nombre d'enregistrements avec station 'non concerné' : ", len(df_non_concerne))

df_concerne_max_1 = df_concerne[df_concerne.somme == maxi-1]
print("nombre d'enregistrements avec ", maxi-1, " erreurs : ", len(df_concerne_max_1))
df_concerne_max_1.loc[:, ('id_pdc_itinerance', 'id_station_itinerance')]

nombre d'enregistrements avec station 'non concerné' :  89
nombre d'enregistrements avec  8  erreurs :  6


,id_pdc_itinerance,id_station_itinerance
21818,FRH14P59663001,FRH14P59663001
21831,FRH14P59309001,FRH14P59309001
35098,FRH14E593090012,FRH14P59309001
35099,FRH14E593090011,FRH14P59309001
35100,FRH14E596630012,FRH14P59663001
35101,FRH14E596630011,FRH14P59663001


In [24]:
FRH14P59309001 = data2[data2.id_station_itinerance == 'FRH14P59309001']
FRH14P59309001.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'adresse_station', 
                          'contact_operateur', 'nom_enseigne', 'implantation_station', 'nbre_pdc', 'horaires', 
                          'condition_acces', 'station_deux_roues')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,adresse_station,contact_operateur,nom_enseigne,implantation_station,nbre_pdc,horaires,condition_acces,station_deux_roues
21831,FRH14P59309001,FRH14P59309001,Hondschoote - Rue de Cassel,"[2.579956, 50.978392]",4 rue de Cassel 59122 Hondschoote,support@passpasselectrique.fr,Pass Pass Electrique,Parking public,1,24/7,Accès libre,false
35098,FRH14E593090012,FRH14P59309001,HONDSCHOOTE - Rue De Cassel,"[2.579956, 50.978392]",4 Rue De Cassel 59122 HONDSCHOOTE,support@alizecharge.fr,pass pass électrique,Voirie,2,Mo-Su 00:00-23:59,Accès libre,FALSE
35099,FRH14E593090011,FRH14P59309001,HONDSCHOOTE - Rue De Cassel,"[2.579956, 50.978392]",4 Rue De Cassel 59122 HONDSCHOOTE,support@alizecharge.fr,pass pass électrique,Voirie,2,Mo-Su 00:00-23:59,Accès libre,FALSE


In [25]:
FRH14P59663001 = data2[data2.id_station_itinerance == 'FRH14P59663001']
FRH14P59663001.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'adresse_station', 
                          'contact_operateur', 'nom_enseigne', 'implantation_station', 'nbre_pdc', 'horaires', 
                          'condition_acces', 'station_deux_roues')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,adresse_station,contact_operateur,nom_enseigne,implantation_station,nbre_pdc,horaires,condition_acces,station_deux_roues
21818,FRH14P59663001,FRH14P59663001,Wormhout -Place de Gaulle,"[2.46799, 50.880845]",92 Place De Gaulle 59470 Wormhout,support@passpasselectrique.fr,Pass Pass Electrique,Parking public,2,24/7,Accès libre,false
35100,FRH14E596630012,FRH14P59663001,WORMHOUT - Place Du General De Gaulle,"[2.467585, 50.883458]",92 Place Du General De Gaulle 59470 WORMHOUT,support@alizecharge.fr,pass pass électrique,Voirie,2,Mo-Su 00:00-23:59,Accès libre,FALSE
35101,FRH14E596630011,FRH14P59663001,WORMHOUT - Place Du General De Gaulle,"[2.467585, 50.883458]",92 Place Du General De Gaulle 59470 WORMHOUT,support@alizecharge.fr,pass pass électrique,Voirie,2,Mo-Su 00:00-23:59,Accès libre,FALSE


# 6 - Exemples d'analyse de relations
## Identification du champ 'root'
Le champ 'root' est l'identifiant d'une ligne du tableau. Dans le cas présent, aucun champ ne remplit ce rôle. Le champ le plus proche est le champ 'id_pdc_itinerance' avec 42 457 valeurs pour 51 276 lignes.

In [44]:
champ = idxs.nindex
root = Iindex.ext(list(range(len(idxs))))

ecart = champ('id_pdc_itinerance').coupling(root)
print('lignes en écart : ',len(ecart))

c = Counter(champ('id_pdc_itinerance').codec).most_common(50)
champ('id_pdc_itinerance').reindex()
print('les 50 pdc avec le plus de lignes : \n', c, '\n')

lignes en écart :  16642
les 50 pdc avec le plus de lignes : 
 [('Non concerné', 519), ('FR55CP84000', 50), ('FR55CP92140', 29), ('FR55CP78140', 28), ('FR55CP83310', 23), ('FR55CP94340', 18), ('FR55CP92260', 16), ('FR55CP78000', 16), ('FR55CP92380', 12), ('FR55CP92130', 12), ('FR55CP83120', 10), ('FRA68P680210015', 10), ('FR55CP83990', 10), ('FR55CP83350', 9), ('FR55CPBP73800', 8), ('FR55CPBP73190', 8), ('FR55CP92280', 6), ('FR55CP30133', 6), ('FR55CPBP514', 6), ('FR55CP33240', 6), ('FR55CPBP698', 6), ('FR55CP92320', 6), ('FRLMSE117226622', 5), ('FRLMSE117201901', 5), ('FRLMSE117226621', 5), ('FRLMSE117228771', 5), ('FRLMSE117201891', 5), ('FRLMSP117226962', 5), ('FRLMSE117228772', 5), ('FRLMSP117226961', 5), ('FRFASE3302702', 4), ('FRS33EMB3315611', 4), ('FRS33EMB3315811', 4), ('FRFASE3302508', 4), ('FRFASE3302405', 4), ('FRFASE3302204', 4), ('FRFASE3302201', 4), ('FRFASE3302504', 4), ('FRFASE3302803', 4), ('FRFASE3302506', 4), ('FRFASE3302802', 4), ('FRS33EMB3315211', 4), ('FRFASE330

In [45]:
FRFASE3302502 = data[data.id_pdc_itinerance == 'FRFASE3302502']
FRFASE3302502.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
1991,FRFASE3302502,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
2122,FRFASE3302502,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2175,FRFASE3302502,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.627000
42539,FRFASE3302502,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:36:39.503000


In [46]:
FRFASE33025 = data[data.id_station_itinerance == 'FRFASE33025']
FRFASE33025.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
1957,FRFASE3302508,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1958,FRFASE3302503,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1959,FRFASE3302507,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1960,FRFASE3302506,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1961,FRFASE3302505,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1962,FRFASE3302504,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1968,FRFASE3302501,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000
1991,FRFASE3302502,FRFASE33025,Aire de Saint Ambreuil,"[4.844517, 46.69318]",8,2023-03-03T17:40:02.071000


In [47]:
FRFASP33025 = data[data.id_station_itinerance == 'FRFASP33025']
FRFASP33025.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
2122,FRFASE3302502,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2131,FRFASE3302501,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2133,FRFASE3302503,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2134,FRFASE3302505,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2135,FRFASE3302506,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2136,FRFASE3302507,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2137,FRFASE3302508,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2141,FRFASE3302504,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.927000
2164,FRFASE3302503,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.627000
2166,FRFASE3302501,FRFASP33025,Aire de Saint Ambreuil,"[4.844761236754013, 46.69092869473468]",8,2023-03-03T17:39:56.627000


In [42]:
FR55CP92140 = data[data.id_pdc_itinerance == 'FR55CP92140']
FR55CP92140.loc[:, ('id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified')]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
2388,FR55CP92140,FR55CP92140,POMPIDOU - CLAMART,"[2.23637, 48.784747]",1,2023-03-03T17:39:50.555000
2407,FR55CP92140,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,"[2.255335, 48.786899]",1,2023-03-03T17:39:50.555000
2408,FR55CP92140,FR55CP92140,VICTOR HUGO - CLAMART,"[2.267296, 48.804229]",1,2023-03-03T17:39:50.555000
2409,FR55CP92140,FR55CP92140,VICTOR HUGO - CLAMART,"[2.267296, 48.804229]",1,2023-03-03T17:39:50.555000
2410,FR55CP92140,FR55CP92140,VICTOR HUGO - CLAMART,"[2.267296, 48.804229]",1,2023-03-03T17:39:50.555000
2411,FR55CP92140,FR55CP92140,VICTOR HUGO - CLAMART,"[2.267296, 48.804229]",1,2023-03-03T17:39:50.555000
2419,FR55CP92140,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,"[2.255335, 48.786899]",1,2023-03-03T17:39:50.555000
2420,FR55CP92140,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,"[2.255335, 48.786899]",1,2023-03-03T17:39:50.555000
2421,FR55CP92140,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,"[2.255335, 48.786899]",1,2023-03-03T17:39:50.555000
2423,FR55CP92140,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,"[2.255335, 48.786899]",1,2023-03-03T17:39:50.555000


- le principal écart est lié à la présence de l'historique des modifications.
- l'écart résiduel (1068 lignes) est à traiter de façon spécifique -> cf exemples d'écarts ci-dessous

In [28]:
id_pdc = Iindex.merging([champ('id_pdc_itinerance'), champ('last_modified')], 'id_pdc')
ecart = id_pdc.coupling(root)
print('lignes en écart : ',len(ecart))

c = Counter(id_pdc.codec).most_common(5)
id_pdc.reindex()
print('les 5 pdc avec le plus de lignes : \n', c, '\n')

lignes en écart :  1068
les 5 pdc avec le plus de lignes : 
 [(('FR55CP84000', '2023-03-03T17:39:50.555000'), 50), (('Non concerné', '2023-03-03T17:37:07.612000'), 34), (('FR55CP92140', '2023-03-03T17:39:50.555000'), 29), (('FR55CP78140', '2023-03-03T17:39:50.555000'), 28), (('FR55CP83310', '2023-03-03T17:39:50.555000'), 23)] 



In [29]:
data.loc[ecart,['id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified']]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
3773,FRP07E6748200312,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",1,2023-03-03T17:39:32.426000
3772,FRP07E6748200312,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",41,2023-03-03T17:39:32.426000
2437,FR55CP84310,FR55CP84310,AVENUE JEAN MONNET - MORIERE,"[4.899412, 43.940189]",1,2023-03-03T17:39:50.555000
2279,FR55CP84310,FR55CP84310,PLACE EMMANUEL VITRIA - MORIERE,"[4.90471, 43.942662]",1,2023-03-03T17:39:50.555000
2540,FR55CP84310,FR55CP84310,PLACE EMMANUEL VITRIA - MORIERE,"[4.90471, 43.942662]",1,2023-03-03T17:39:50.555000
...,...,...,...,...,...,...
8945,FRP07E6748200334,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",1,2023-03-03T17:39:32.426000
7890,FRP07E6748200335,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",1,2023-03-03T17:39:32.426000
7882,FRP07E6748200335,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",41,2023-03-03T17:39:32.426000
21803,FRS87EMB872412,FRS87PMB872412,SAINT-PRIEST-TAURION - ESPLANADE DU STADE,"[1.330306, 45.875523]",2,2023-03-03T17:38:51.190000


- l'élimination des doublons (en conservant la date de modification la plus résente) permet d'avoir une ligne par point de charge.

In [30]:
data2 = data.sort_values(by='last_modified').drop_duplicates('id_pdc_itinerance', keep='last').reset_index(drop=True)
print('data2 : ', len(data2))

data2 :  42457


In [31]:
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen))

idxs (len, lenlidx, sumcodec) :  42457 50 193355


In [32]:
champ = idxs2.nindex
root = Iindex.ext(list(range(len(idxs))))
print("Couplage entre le champ 'id_pdc_itinerance' et la liste des lignes : ", champ('id_pdc_itinerance').iscoupled(root))

Couplage entre le champ 'id_pdc_itinerance' et la liste des lignes :  False


## Représentation des stations

- deux champs sont candidats pour identifier les stations : 'id_station_itinerance', 'nom_station'
- Le champ le plus approprié est 'id_station_itinerance' (plus grand nombre de valeurs différentes)
- une autre solution pourrait être de considérer qu'une station est identifiée par le couple 'id_station_itinerance', 'nom_station' mais ceci conduit à doublonner une station pour laquelle une erreur de nom a été commise (cf exemple ci-dessous).

In [ ]:
print('le nom dérive de la station : ', champ('id_station_itinerance').isderived(champ('nom_station')), '\n')

ecart_nom = champ('id_station_itinerance').coupling(champ('nom_station'))
c_nom = Counter(champ('id_station_itinerance').codec).most_common(5)
champ('id_station_itinerance').reindex()

print('nombre de lignes en écart : ',len(ecart_nom))
print('les 5 stations avec le plus de noms : \n', c_nom, '\n')

In [ ]:
print('stations avec plusieurs noms : \n')
data2.loc[ecart_nom,['id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY']]

## localisation des stations
Les stations sont localisées au travers d'une coordonnée géographique et une adresse complète cette localisation. De même que pour la dénomination des stations, deux questions peuvent être posées :
- peut-on avoir plusieurs stations avec la même coordonnée ?
- peut-on avoir plusieurs coordonnées pour une même station ?

L'analyse ci-dessous montre que des erreurs d'identification de stations conduisent à répondre oui à ces deux questions.

In [ ]:
ecart_XY = champ('id_station_itinerance').coupling(champ('coordonneesXY'))
c_XY = Counter(champ('id_station_itinerance').codec).most_common(5)
champ('id_station_itinerance').reindex()

print('nombre de lignes en écart : ',len(ecart_XY))

print('les 5 stations avec le plus de positions différentes : \n', c_XY, '\n')

In [ ]:
print('stations avec plusieurs coordonnées : \n')
data2.loc[ecart_XY,['id_station_itinerance', 'nom_station', 'coordonneesXY']]

In [ ]:
ecart_stat = champ('coordonneesXY').coupling(champ('id_station_itinerance'))
c_stat = Counter(champ('coordonneesXY').codec).most_common(5)
champ('coordonneesXY').reindex()

print('nombre de lignes en écart : ',len(ecart_stat))

print('les 5 positions avec le plus grand nombre de stations associées : \n', c_stat, '\n')

In [ ]:
print('positions avec plusieurs stations associées : \n')
data2.loc[ecart_stat,['coordonneesXY', 'id_station_itinerance', 'nom_station']]

-----------------------
# Annexe : Initialisation du modèle de données 'mermaid'

In [1]:
from base64 import b64encode
from IPython.display import Image, display

def mermaid(graph):
  display(Image(url="https://mermaid.ink/img/" + b64encode(graph.encode("ascii")).decode("ascii")))

In [2]:
# modèle de données suivant l'application 'mermaid' 
irve = """ 
erDiagram
    AMENAGEUR ||..|{ STATION : amenage
    AMENAGEUR {
        string nom_amenageur
        string siren_amenageur
        string contact_amenageur 
    }
    OPERATEUR ||..|{ STATION : "exploite pour le compte de l enseigne"
    OPERATEUR {
        string contact_operateur PK "M"
        string nom_operateur 
        string telephone_operateur 
    }
    ENSEIGNE ||..|{ STATION : "heberge"
    ENSEIGNE {
        string nom_enseigne PK "M" 
    }
    STATION {
        string  id_station_itinerance PK "M"
        string  nom_station "M"
        enum    implantation_station "M"
        integer nbre_pdc "M"
        string  condition_acces "M"
        string  horaires "M"
        boolean station_deux_roues "M"
        string  id_station_local
        enum    raccordement
        string  num_pdl
        date    date_mise_en_service 
    }
    LOCALISATION ||--|{ STATION : "localise"
    LOCALISATION {
       array   coordonneesXY PK "M"
       string  adresse_station "M"
       string  code_insee_commune 
    }
    STATION ||--|{ POINT_DE_CHARGE : regroupe
    POINT_DE_CHARGE {
        string id_pdc_itinerance PK "M Root"
        number puissance_nominale "M"
        boolean prise_type_ef "M"
        boolean prise_type_2 "M"
        boolean prise_type_2combo_ccs "M"
        boolean prise_type_chademo "M"
        boolean prise_type_autre "M"
        boolean paiement_acte "M"
        boolean paiement_autre "M"
        boolean reservation "M"
        enum    accessibilite_pmr "M"
        string  restriction_gabarit "M"
        date    date_maj "M"
        string  id_pdc_local
        boolean gratuit
        boolean paiement_cb
        string  tarification
        string  observations
        boolean cable_t2_attache 
    }
"""